In [26]:
import pandas as pd
import numpy as np

import yfinance as yf
import plotly.graph_objects as go

In [27]:
from urllib.request import urlretrieve
from urllib.parse import urlencode
import requests

import pandas as pd

ticker = 'X:XRPUSD'
range='1'
interval='day'
start='2021-04-12'
end='2023-03-12'
limit=2000000000
polygon_api_key = "R1ElcqjoruJBGIsBPUnhvEw2jOZjM6c8"

baseurl = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{range}/{interval}/{start}/{end}?adjusted=true&sort=asc&limit={limit}&apiKey={polygon_api_key}'

req = requests.get(baseurl)
print(req.json())

{'ticker': 'X:XRPUSD', 'queryCount': 700, 'resultsCount': 700, 'adjusted': True, 'results': [{'v': 322477870.9620089, 'vw': 1.3716, 'o': 1.35118, 'c': 1.469267, 'h': 1.47648, 'l': 1.31763, 't': 1618185600000, 'n': 134318}, {'v': 591832601.0049697, 'vw': 1.6882, 'o': 1.4691, 'c': 1.79604, 'h': 1.88982, 'l': 1.41769, 't': 1618272000000, 'n': 340463}, {'v': 556151416.6015683, 'vw': 1.7817, 'o': 1.79693, 'c': 1.8346, 'h': 1.96868, 'l': 1.5454, 't': 1618358400000, 'n': 330059}, {'v': 261305749.0306577, 'vw': 1.7544, 'o': 1.8348, 'c': 1.757115, 'h': 1.88545, 'l': 1.6486, 't': 1618444800000, 'n': 196079}, {'v': 433212558.0284068, 'vw': 1.6139, 'o': 1.7589, 'c': 1.5494, 'h': 1.81175, 'l': 1.41251, 't': 1618531200000, 'n': 298326}, {'v': 237965128.943195, 'vw': 1.6225, 'o': 1.5475, 'c': 1.539224, 'h': 1.74524, 'l': 1.52117, 't': 1618617600000, 'n': 162844}, {'v': 515450104.52200484, 'vw': 1.3196, 'o': 1.54127, 'c': 1.4076, 'h': 1.56714, 'l': 1.1197, 't': 1618704000000, 'n': 328410}, {'v': 31802

In [28]:
df = pd.DataFrame(req.json()['results'])

import datetime

def convert_timestamp(timestamp):
    return datetime.datetime.fromtimestamp(timestamp / 1000.0).strftime('%Y-%m-%d %H:%M:%S.%f')

df['t'] = df['t'].apply(convert_timestamp)

df = df.rename({
    'o': 'Open',
    'c': 'Close',
    'h': 'High',
    'l': 'Low',
    't': 'Datetime'
}, axis=1)

ticker_data = df.loc[:, ['Open', 'Close', 'High', 'Low', 'Datetime']]
ticker_data = ticker_data.set_index('Datetime')

In [29]:
class Backtest:
    balance = 0

    stoploss = 0
    target = 0

    position = {
        "entry_date": None,
        "entry_price": None,
        "exit_price": None,
        "exit_date": None,
        "target": None,
        "stoploss": None,
        "PnL": None,
        "crossover_type": None,
        "active": False,
        "size": None,
        "cost": None,
        "balance": None,
    }

    trade_book = pd.DataFrame(
        columns=[
            "entry_date",
            "entry_price",
            "exit_price",
            "exit_date",
            "target",
            "stoploss",
            "PnL",
            "crossover_type",
            "active",
            "size",
            "margin_used",
            "balance",
            "balance_on_open"
        ]
    )

    def __init__(self, balance, stoploss, reward_ratio):
        self.balance = balance
        self.stoploss = stoploss
        self.target = stoploss * reward_ratio        
        
        self.clear_position()
        self.reset_tradebook()

    def clear_position(self):
        self.position["entry_date"] = None
        self.position["entry_price"] = None
        self.position["exit_price"] = None
        self.position["exit_date"] = None

        self.position["target"] = None
        self.position["stoploss"] = None

        self.position["PnL"] = None
        self.position["crossover_type"] = None

        self.position["active"] = False
        self.position["size"] = 0
        self.position["balance"] = None

        self.position["status"] = None


    def reset_tradebook(self):
        self.trade_book = pd.DataFrame(
            columns=[
                "entry_date",
                "entry_price",
                "exit_price",
                "exit_date",
                "target",
                "stoploss",
                "PnL",
                "crossover_type",
                "active",
                "size",
                "margin_used",
                "balance",
                "balance_on_open"
            ]
        )
        
        
    def open(self, entry_date, entry_price, stoploss, target, size, crossover_type):
        if self.balance is None or self.balance == 0:
            raise Exception("Oops, you ran out of bucks.")
        
        if self.balance < entry_price*size:
            print("Balance: ", self.balance)
            print("Lot Size: ", size)
            print("Ticket Price: ", entry_price*size)
            raise Exception("Oops, you are low on bucks.")

        self.position["active"] = True

        self.position["entry_date"] = entry_date
        self.position["entry_price"] = entry_price

        self.position["stoploss"] = stoploss
        self.position["target"] = target

        self.position["size"] = size
        self.position["crossover_type"] = crossover_type
        self.position["margin_used"] = entry_price * size
        self.position["balance_on_open"] = self.balance - entry_price * size 

        self.balance = self.balance - entry_price * size

    def close(self, candle, exit_price, exit_date):
        # Closes the trade / position
        self.position["active"] = False
        self.position["exit_date"] = candle.name
        self.position["exit_price"] = exit_price
        self.position["PnL"] = (
            exit_price - self.position["entry_price"]
        ) * self.position["size"]
        
        if self.position["crossover_type"] == "short":
            self.position["PnL"] = -1*self.position["PnL"]

        balance = self.position['balance_on_open'] + \
            self.position['margin_used'] + self.position['PnL'] 
        self.position["balance"] = balance
        self.balance = balance

        row = pd.Series(self.position)
        self.trade_book.loc[len(self.trade_book)] = row

        self.clear_position()

    def strategy():
        ## to be implemented in subclass
        pass

    def print(self):
        print(self.balance, self.stoploss, self.target)

In [30]:
class Tester(Backtest):
    ticker_data = None
    size = 10

    def __init__(self, balance, stoploss, reward_ratio):
        super().__init__(balance, stoploss, reward_ratio)
        self.ticker_data = None
        self.reset_tradebook()
        
        self.profitable_trades = None
        self.losing_trades = None

    def init(self, ticker_data, size, small_ema, large_ema):
        ticker_data["small_ema"] = (
            ticker_data["Close"].ewm(span=small_ema, adjust=False).mean()
        )

        ticker_data["large_ema"] = (
            ticker_data["Close"].ewm(span=large_ema, adjust=False).mean()
        )

        ticker_data["crossover_pos"] = np.where(
            ticker_data["small_ema"] > ticker_data["large_ema"], 1, 0
        )

        ticker_data["crosspoint"] = np.where(
            ticker_data["crossover_pos"] != ticker_data["crossover_pos"].shift(1),
            True,
            False,
        )
        
        self.small_ema = small_ema
        self.large_ema = large_ema
        self.ticker_data = None
        self.ticker_data = ticker_data
        self.size = size

    def chart(self, small_ema, large_ema):
        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=ticker_data.index,
                    open=ticker_data["Open"],
                    high=ticker_data["High"],
                    low=ticker_data["Low"],
                    close=ticker_data["Close"],
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["small_ema"],
                    name=f"small_ema_{small_ema}",
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["large_ema"],
                    name=f"large_ema_{large_ema}",
                ),
            ]
        )

        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.show()

    def print(self):
        print(self.position)

    def run(self):
        if self.ticker_data is None:
            raise Exception("No ticker data to run backtest on")
        
        # Skip first 200 candles
        start_index = self.large_ema

        while start_index < len(self.ticker_data):
            self.strategy(self.ticker_data.iloc[start_index])
            start_index += 1

    def stats(self):
        print("Total Trades: ", len(self.trade_book))
        print("PnL: ", self.trade_book["PnL"].sum())

        total_trades = len(self.trade_book)
        profitable_trades = self.trade_book[self.trade_book["PnL"] > 0]
        losing_trades = self.trade_book[self.trade_book["PnL"] < 0]
        win_percentage = len(profitable_trades) / total_trades * 100
        lose_percentage = len(losing_trades) / total_trades * 100
        
        
        print("Profitable Trades: ", len(profitable_trades))
        print("Losing Trades: ", len(losing_trades))
        print("Win Percentage: ", win_percentage)
        print("Lose Percentage: ", lose_percentage)

        self.profitable_trades = profitable_trades
        self.losing_trades = losing_trades

    def strategy(self, candle):
        if self.position["active"] == True:
            # check if target met or stoploss hit
            # calculate trade results, PnL
            # close position

            if self.position["crossover_type"] == "long":
                if candle["Close"] < self.position["stoploss"]:
                    self.close(candle, self.position["stoploss"], candle.name)
                    return

                if candle["Close"] > self.position["target"]:
                    self.close(candle, self.position["target"], candle.name)
                    return

            if self.position["crossover_type"] == "short":
                if candle["Close"] > self.position["stoploss"]:
                    self.close(candle, self.position["stoploss"], candle.name)
                    return

                if candle["Close"] < self.position["target"]:
                    self.close(candle, self.position["target"], candle.name)
                    return

            return

        if candle["crosspoint"] == True:
            entry_price = candle["Close"]
            crossover_type = "long" if candle["crossover_pos"] == 1 else "short"
            stoploss = None
            target = None
            
            if crossover_type == "short":
                stoploss = entry_price + self.stoploss
                target = entry_price - self.target
                
            if crossover_type == "long":
                stoploss = entry_price - self.stoploss
                target = entry_price + self.target

            self.open(
                entry_date=candle.name,
                entry_price=entry_price,
                stoploss=stoploss,
                target=target,
                size=self.size,
                crossover_type=crossover_type,
            )

In [31]:
small_ema = 21
large_ema = 55

# balance, stoploss, reward_ratio
ttr = Tester(4000, 0.001, 10)
# ticker_data, ticker_size,  ticket_size, small_ema, large_ema
ttr.init(ticker_data, 2000, small_ema, large_ema)
# small_ema, large_ema
ttr.chart(small_ema, large_ema)
# run backtesting
ttr.run()
ttr.stats()

t_book2 = ttr.trade_book
profitables = ttr.profitable_trades
losers = ttr.losing_trades

Total Trades:  10
PnL:  68.00000000000051
Profitable Trades:  4
Losing Trades:  6
Win Percentage:  40.0
Lose Percentage:  60.0
